### Load Data

In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from single_split_gridsearch import SingleSplitGridSearch

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#10000
n = 5000

In [2]:
videos_train = ['MVI_0788_VIS_OB', 'MVI_0789_VIS_OB', 'MVI_0792_VIS_OB', 'MVI_0794_VIS_OB', 'MVI_0796_VIS_OB']
video_test = 'MVI_0804_VIS_OB'

# Reads bank currency note data into pandas dataframe - TRAIN
bankdata_train = pd.read_csv("../METEP/Features/" + videos_train[0] + "R_frame0.csv")

for i in range (5):
    k = 0
    print(i)
    if i == 0:
        k = 20
    while k < 299:
        print(k)
        bankdata_train = bankdata_train.append(pd.read_csv("../METEP/Features/" + videos_train[i] + "R_frame" + str(k) + ".csv"))
        k += 20
        if videos_train[i] == 'MVI_0789_VIS_OB' and k == 280:
            k = 400

# Show number of lines and columns
#bankdata.shape
# Show a able wih some lines of he daa
#bankdata.head()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
1
0
20
40
60
80
100
120
140
160
180
200
220
240
260
2
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
3
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
4
0
20
40
60
80
100
120
140
160
180
200
220
240
260
280


In [3]:
# Reads bank currency note data into pandas dataframe - TEST

bankdata_test = pd.read_csv("../METEP/Features/" + video_test + "R_frame0.csv")

k = 20
while k < 299:
    print(k)
    bankdata_test = bankdata_test.append(pd.read_csv("../METEP/Features/" + video_test + "R_frame" + str(k) + ".csv"))
    k += 20
    if video_test == 'MVI_0789_VIS_OB' and k == 280:
            k = 400

20
40
60
80
100
120
140
160
180
200
220
240
260
280


In [4]:
# Divide the data into attributes and labels

# Store attributes
X_train = bankdata_train.drop('class', axis=1)
# Store labels
Y_train = bankdata_train['class']

# Store attributes
X_test = bankdata_test.drop('class', axis=1)
# Store labels
Y_test = bankdata_test['class']

### Kmeans

In [5]:
# Normalizando as bases

from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [6]:
from sklearn.cluster import MiniBatchKMeans

mask_Sky = Y_train == 1
mask_NonSky = Y_train == 0

km = MiniBatchKMeans(
    n_clusters=n, init='k-means++',
    n_init=10, max_iter=300, 
    tol=1e-04
)

Sky_km = km.fit(X_train[mask_Sky])
sky = km.cluster_centers_

NonSky_km = km.fit(X_train[mask_NonSky])
nonsky = km.cluster_centers_

features = np.concatenate([sky, nonsky])

class_Sky = np.ones((n), dtype=np.int8)
class_NonSky = np.zeros((n), dtype=np.int8)

classes = np.concatenate([class_Sky, class_NonSky])

c:\users\thais\appdata\local\programs\python\python36\lib\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)
c:\users\thais\appdata\local\programs\python\python36\lib\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)
c:\users\thais\appdata\local\programs\python\python36\lib\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)
c:\users\thais\appdata\local\programs\python\python36\lib\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it to 3*k
  init_size=init_size)
c:\users\thais\appdata\local\programs\python\python36\lib\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=5000. Setting it 

------------

### SingleSplitGridSearch

In [7]:
#Inicializar uma instância da SVM. Os parametros que não forem alterados durante
#o grid search permanecem os mesmos em todos os modelos avaliados.
#Por exemplo, todos os modelos avaliados terão probability=True
svm = SVC(probability=True)

#Indicar quais parametros serão avaliados na busca exaustiva (GridSearch)
Cs = [1, 10, 100, 1000]
gammas = [2e-3, 2e-2, 2e-1, 'auto']

#Param_grid é um dicionário que associa o nome dos parametros passados para criação do classificador
#(parametros passados para o SVC, por exemplo) a lista de parametros a serem avaliados.
param_grid={
    'C' : Cs,
    'gamma' : gammas
}

#Configurar a busca exaustiva. Note que é passado um parametro test_size, que indica a proporção do conjunto
#de treino que será usado para validação. Neste caso, 20% dos dados de treino passados para o método fit
#serão usados como validação. Além disto, esta separação é feita de forma estratificada, tentando manter
#a mesma proporção de dados no treino e na validação de acordo com cada classe.
#n_jobs permite que cada modelo seja treinado em paralelo, neste caso, n_jobs=4 permite que 4
#modelos sejam treinados em paralelo. verbose=10 indica que serão mostradas informações sobre a execução
#do treino. Caso não queira estas informações, basta atribuir verbose=0.
model = SingleSplitGridSearch(svm, param_grid, n_jobs=2, verbose=10, test_size=0.2)

#Fazer a busca exaustiva. Ao final, model será retreinado com a melhor combinação de parametros obtida, usando
#todos os dados de treino.
model.fit(features, classes)

#Por curiosidade, mostrar os melhores parametros encontrados
print()
print("Melhores parametros encontrados: " + str(model.best_params_))
print()

#O método predict está disponível
predicted = model.predict(X_test)
print(classification_report(Y_test, predicted))

#E o predict_proba tbm...
model.predict_proba(X_test)[0]

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   44.6s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:  2.6min
[Parallel(n_jobs=2)]: Done  16 out of  16 | elapsed:  7.0min finished



Melhores parametros encontrados: {'C': 1000, 'gamma': 0.2}

              precision    recall  f1-score   support

           0       1.00      0.78      0.88   1340072
           1       0.02      0.68      0.04      9928

    accuracy                           0.78   1350000
   macro avg       0.51      0.73      0.46   1350000
weighted avg       0.99      0.78      0.87   1350000



array([0.71070631, 0.28929369])